In [ ]:
import osmnx as ox
import geopandas as gpd

In [ ]:
# include the tag we need
ox.utils.config(useful_tags_node=['ref', 'highway','traffic_signals:direction', 'direction','crossing','stop','stop:direction'])

In [ ]:
utn = ox.settings.useful_tags_node
utn

In [ ]:
file_path = '../Data/'
output_file_path = file_path + 'Output/'

# Import human settlement shapefile from GHS UCDB
Source: GHS Urban Centre Database 2015
https://ghsl.jrc.ec.europa.eu/download.php?ds=ucdb 

In [ ]:
UCDB_gdf = gpd.read_file(file_path + "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg")

In [ ]:
# Select the polygon that is the human settlement area of LA: plot it, you can see there are holes in it
LA_UCDB_gdf = UCDB_gdf[UCDB_gdf['UC_NM_MN'] == 'Los Angeles']
LA_UCDB_gdf.plot()

# Import TIGERline census tract shapefile

In [ ]:
# census tracts in California
CA_22tract = gpd.read_file(file_path + "tl_2022_06_tract/tl_2022_06_tract.shp")
LA_UCDB_gdf.crs == CA_22tract.crs

In [ ]:
# project the CRS to be the same - 4326
CA_22tract_crs = CA_22tract.to_crs(4326)
LA_UCDB_gdf.crs == CA_22tract_crs.crs

In [ ]:
# selecting census tracts in LA county
LA_22tract = CA_22tract_crs[CA_22tract_crs['COUNTYFP'] == '037']

# Get the shapefile of intersection of LA county and LA human settlement area

In [ ]:
# Plot the difference of two shapefile
ax = LA_22tract.plot(color="palegreen", edgecolor="green", figsize=(20, 10))
LA_UCDB_gdf.plot(ax=ax, color="red")

In [ ]:
# Get the intersection proportion of these two shapefiles
LA_clip = gpd.clip(LA_22tract, LA_UCDB_gdf)
LA_clip_whole = LA_clip.dissolve()

In [ ]:
LA_clip_whole.plot()

# Get the convex hull of it

In [ ]:
LA_clip_convex = LA_clip_whole.convex_hull

In [ ]:
# plot the convex hull of the intersected shapefile, which is our study area
ax = LA_clip_convex.plot(color="palegreen", edgecolor="green", figsize=(20, 10))
LA_clip_whole.plot(ax=ax, color="red")

In [ ]:
LA_clip_convex.to_file(output_file_path + 'LA_clip_convex.shp')

# get street network for analysis
### Using the above convex hull shapefile are our study region
### Get strongly connected graph

In [ ]:
LA_clip_convex_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(LA_clip_convex))
LA_clip_convex_polygon = LA_clip_convex_gdf.iloc[0]['geometry']

In [ ]:
G = ox.graph_from_polygon(LA_clip_convex_polygon, simplify=False, network_type='drive')
ox.plot_graph(G)

In [ ]:
G.nodes(data=True)[5432488414]

In [ ]:
# Save Geopackages
ox.save_graph_geopackage(G, filepath=output_file_path + 'LA_clip_convex_network_non_simplify_all_direction.gpkg')
# Save Graphml
ox.save_graphml(G, filepath=output_file_path + 'LA_clip_convex_network_non_simplify_all_direction.graphml')

In [ ]:
# get strongly connected graph
G = ox.utils_graph.get_largest_component(G, strongly=True)

In [ ]:
# Save Geopackages
ox.save_graph_geopackage(G, filepath=output_file_path + 'LA_clip_convex_strong_network_non_simplify_all_direction.gpkg')
# Save Graphml
ox.save_graphml(G, filepath=output_file_path + 'LA_clip_convex_strong_network_non_simplify_all_direction.graphml')